In [1]:
from common import *

In [2]:
fnames = glob('../data/lingvo/raw/*.txt')

In [3]:
dim = 300

In [4]:
import gc, io

def process(fnames, i):
    with io.open('../data/lingvo/%s.txt'%i, 'w', encoding='utf8') as fw:
        for fn in fnames:
            with io.open(fn, encoding='utf8') as fr:
                text = fr.read()
            sents = sent_tokenize(text)
            sents = [[w for w in word_tokenize(s) if w not in stop_list and len(w)>1] for s in sents]
            sents = [s for s in sents if len(s)]
            s = json.dumps((basename(fn).split('.')[0], sents))            
            fw.write(s + u'\n')
    gc.collect()


In [ ]:
parallelizer = Parallel(n_jobs=cpu_count)

# this iterator returns the functions to execute for each task
tasks_iterator = ( delayed(process)(list_block, i) for 
                  i, list_block in enumerate(grouper(len(fnames)//500, fnames)) ) 
result = parallelizer( tasks_iterator )

In [19]:
import ujson

def iter_sents():
    with GzipFile('../data/lingvo/corpus_json.txt.gz', 'r') as fr:
        for line in fr:
            _id, sents = ujson.loads(line)
            for s in sents:
                yield s
                
class Sentences(object):
    def __iter__(self):
        for sent in iter_sents():
            yield sent

In [ ]:
bigram = gensim.models.Phrases(iter_sents())

2017-10-06 23:31:37,839 [MainThread  ] [INFO ]  collecting all words and their counts
2017-10-06 23:31:37,871 [MainThread  ] [INFO ]  PROGRESS: at sentence #0, processed 0 words and 0 word types
2017-10-06 23:31:38,283 [MainThread  ] [INFO ]  PROGRESS: at sentence #10000, processed 132059 words and 71990 word types
2017-10-06 23:31:38,618 [MainThread  ] [INFO ]  PROGRESS: at sentence #20000, processed 254236 words and 133130 word types
2017-10-06 23:31:39,015 [MainThread  ] [INFO ]  PROGRESS: at sentence #30000, processed 400912 words and 198034 word types
2017-10-06 23:31:39,345 [MainThread  ] [INFO ]  PROGRESS: at sentence #40000, processed 523348 words and 258034 word types
2017-10-06 23:31:39,728 [MainThread  ] [INFO ]  PROGRESS: at sentence #50000, processed 664973 words and 315634 word types
2017-10-06 23:31:40,117 [MainThread  ] [INFO ]  PROGRESS: at sentence #60000, processed 799556 words and 374368 word types
2017-10-06 23:31:40,434 [MainThread  ] [INFO ]  PROGRESS: at sentenc

2017-10-06 23:32:00,266 [MainThread  ] [INFO ]  PROGRESS: at sentence #650000, processed 7926933 words and 2968697 word types
2017-10-06 23:32:00,640 [MainThread  ] [INFO ]  PROGRESS: at sentence #660000, processed 8068470 words and 3002081 word types
2017-10-06 23:32:00,986 [MainThread  ] [INFO ]  PROGRESS: at sentence #670000, processed 8190606 words and 3042769 word types
2017-10-06 23:32:01,324 [MainThread  ] [INFO ]  PROGRESS: at sentence #680000, processed 8311393 words and 3080963 word types
2017-10-06 23:32:01,650 [MainThread  ] [INFO ]  PROGRESS: at sentence #690000, processed 8427738 words and 3116362 word types
2017-10-06 23:32:01,961 [MainThread  ] [INFO ]  PROGRESS: at sentence #700000, processed 8539219 words and 3151099 word types
2017-10-06 23:32:02,312 [MainThread  ] [INFO ]  PROGRESS: at sentence #710000, processed 8667346 words and 3189294 word types
2017-10-06 23:32:02,656 [MainThread  ] [INFO ]  PROGRESS: at sentence #720000, processed 8789247 words and 3229578 wor

2017-10-06 23:32:22,525 [MainThread  ] [INFO ]  PROGRESS: at sentence #1300000, processed 15832295 words and 5290616 word types
2017-10-06 23:32:22,866 [MainThread  ] [INFO ]  PROGRESS: at sentence #1310000, processed 15954695 words and 5326840 word types
2017-10-06 23:32:23,217 [MainThread  ] [INFO ]  PROGRESS: at sentence #1320000, processed 16082510 words and 5359050 word types
2017-10-06 23:32:23,605 [MainThread  ] [INFO ]  PROGRESS: at sentence #1330000, processed 16224274 words and 5395267 word types
2017-10-06 23:32:23,985 [MainThread  ] [INFO ]  PROGRESS: at sentence #1340000, processed 16360920 words and 5426886 word types
2017-10-06 23:32:24,311 [MainThread  ] [INFO ]  PROGRESS: at sentence #1350000, processed 16475278 words and 5458937 word types
2017-10-06 23:32:24,676 [MainThread  ] [INFO ]  PROGRESS: at sentence #1360000, processed 16607770 words and 5494173 word types
2017-10-06 23:32:25,009 [MainThread  ] [INFO ]  PROGRESS: at sentence #1370000, processed 16725768 words

2017-10-06 23:32:45,254 [MainThread  ] [INFO ]  PROGRESS: at sentence #1950000, processed 23884672 words and 7380231 word types
2017-10-06 23:32:45,631 [MainThread  ] [INFO ]  PROGRESS: at sentence #1960000, processed 24016635 words and 7410045 word types
2017-10-06 23:32:45,970 [MainThread  ] [INFO ]  PROGRESS: at sentence #1970000, processed 24139591 words and 7440417 word types
2017-10-06 23:32:46,300 [MainThread  ] [INFO ]  PROGRESS: at sentence #1980000, processed 24258290 words and 7469633 word types
2017-10-06 23:32:46,630 [MainThread  ] [INFO ]  PROGRESS: at sentence #1990000, processed 24380452 words and 7500512 word types
2017-10-06 23:32:46,960 [MainThread  ] [INFO ]  PROGRESS: at sentence #2000000, processed 24495551 words and 7530655 word types
2017-10-06 23:32:47,306 [MainThread  ] [INFO ]  PROGRESS: at sentence #2010000, processed 24617943 words and 7560345 word types
2017-10-06 23:32:47,648 [MainThread  ] [INFO ]  PROGRESS: at sentence #2020000, processed 24734037 words

2017-10-06 23:33:07,168 [MainThread  ] [INFO ]  PROGRESS: at sentence #2600000, processed 31790234 words and 9251947 word types
2017-10-06 23:33:07,526 [MainThread  ] [INFO ]  PROGRESS: at sentence #2610000, processed 31912783 words and 9278672 word types
2017-10-06 23:33:07,898 [MainThread  ] [INFO ]  PROGRESS: at sentence #2620000, processed 32043199 words and 9304400 word types
2017-10-06 23:33:08,221 [MainThread  ] [INFO ]  PROGRESS: at sentence #2630000, processed 32154750 words and 9330039 word types
2017-10-06 23:33:08,598 [MainThread  ] [INFO ]  PROGRESS: at sentence #2640000, processed 32287893 words and 9359818 word types
2017-10-06 23:33:08,946 [MainThread  ] [INFO ]  PROGRESS: at sentence #2650000, processed 32415186 words and 9389170 word types
2017-10-06 23:33:09,258 [MainThread  ] [INFO ]  PROGRESS: at sentence #2660000, processed 32528288 words and 9414859 word types
2017-10-06 23:33:09,601 [MainThread  ] [INFO ]  PROGRESS: at sentence #2670000, processed 32654847 words

...

2017-10-07 00:24:29,930 [MainThread  ] [INFO ]  PROGRESS: at sentence #83420000, processed 1015931447 words and 25524979 word types
2017-10-07 00:24:30,330 [MainThread  ] [INFO ]  PROGRESS: at sentence #83430000, processed 1016047302 words and 25541632 word types
2017-10-07 00:24:30,728 [MainThread  ] [INFO ]  PROGRESS: at sentence #83440000, processed 1016168177 words and 25558872 word types
2017-10-07 00:24:31,121 [MainThread  ] [INFO ]  PROGRESS: at sentence #83450000, processed 1016291841 words and 25575632 word types
2017-10-07 00:24:31,474 [MainThread  ] [INFO ]  PROGRESS: at sentence #83460000, processed 1016405414 words and 25592209 word types
2017-10-07 00:24:31,852 [MainThread  ] [INFO ]  PROGRESS: at sentence #83470000, processed 1016530891 words and 25609325 word types
2017-10-07 00:24:32,247 [MainThread  ] [INFO ]  PROGRESS: at sentence #83480000, processed 1016661080 words and 25629870 word types
2017-10-07 00:24:32,614 [MainThread  ] [INFO ]  PROGRESS: at sentence #83490

2017-10-07 00:24:53,474 [MainThread  ] [INFO ]  PROGRESS: at sentence #84050000, processed 1023693540 words and 26594426 word types
2017-10-07 00:24:53,827 [MainThread  ] [INFO ]  PROGRESS: at sentence #84060000, processed 1023820850 words and 26610986 word types
2017-10-07 00:24:54,199 [MainThread  ] [INFO ]  PROGRESS: at sentence #84070000, processed 1023949094 words and 26630064 word types
2017-10-07 00:24:54,546 [MainThread  ] [INFO ]  PROGRESS: at sentence #84080000, processed 1024075650 words and 26648597 word types
2017-10-07 00:24:54,921 [MainThread  ] [INFO ]  PROGRESS: at sentence #84090000, processed 1024211733 words and 26664411 word types
2017-10-07 00:24:55,247 [MainThread  ] [INFO ]  PROGRESS: at sentence #84100000, processed 1024326466 words and 26680885 word types
2017-10-07 00:24:55,608 [MainThread  ] [INFO ]  PROGRESS: at sentence #84110000, processed 1024448051 words and 26700028 word types
2017-10-07 00:24:55,962 [MainThread  ] [INFO ]  PROGRESS: at sentence #84120

2017-10-07 00:25:19,745 [MainThread  ] [INFO ]  PROGRESS: at sentence #84680000, processed 1031379975 words and 27652266 word types
2017-10-07 00:25:20,124 [MainThread  ] [INFO ]  PROGRESS: at sentence #84690000, processed 1031508227 words and 27670990 word types
2017-10-07 00:25:20,470 [MainThread  ] [INFO ]  PROGRESS: at sentence #84700000, processed 1031627640 words and 27686672 word types
2017-10-07 00:25:20,834 [MainThread  ] [INFO ]  PROGRESS: at sentence #84710000, processed 1031753786 words and 27704868 word types
2017-10-07 00:25:21,157 [MainThread  ] [INFO ]  PROGRESS: at sentence #84720000, processed 1031864599 words and 27724326 word types
2017-10-07 00:25:21,529 [MainThread  ] [INFO ]  PROGRESS: at sentence #84730000, processed 1031993748 words and 27739128 word types
2017-10-07 00:25:21,874 [MainThread  ] [INFO ]  PROGRESS: at sentence #84740000, processed 1032114348 words and 27756718 word types
2017-10-07 00:25:22,203 [MainThread  ] [INFO ]  PROGRESS: at sentence #84750

2017-10-07 00:25:42,355 [MainThread  ] [INFO ]  PROGRESS: at sentence #85310000, processed 1039023688 words and 28730172 word types
2017-10-07 00:25:42,694 [MainThread  ] [INFO ]  PROGRESS: at sentence #85320000, processed 1039141395 words and 28747055 word types
2017-10-07 00:25:43,035 [MainThread  ] [INFO ]  PROGRESS: at sentence #85330000, processed 1039260689 words and 28763928 word types
2017-10-07 00:25:43,381 [MainThread  ] [INFO ]  PROGRESS: at sentence #85340000, processed 1039380403 words and 28780601 word types
2017-10-07 00:25:43,711 [MainThread  ] [INFO ]  PROGRESS: at sentence #85350000, processed 1039495122 words and 28796567 word types
2017-10-07 00:25:44,062 [MainThread  ] [INFO ]  PROGRESS: at sentence #85360000, processed 1039619001 words and 28811700 word types
2017-10-07 00:25:44,386 [MainThread  ] [INFO ]  PROGRESS: at sentence #85370000, processed 1039731246 words and 28830863 word types
2017-10-07 00:25:44,743 [MainThread  ] [INFO ]  PROGRESS: at sentence #85380

2017-10-07 00:26:04,683 [MainThread  ] [INFO ]  PROGRESS: at sentence #85940000, processed 1046552686 words and 29778275 word types
2017-10-07 00:26:05,101 [MainThread  ] [INFO ]  collected 29797470 word types from a corpus of 1046701979 words (unigram + bigrams) and 85949516 sentences
2017-10-07 00:26:05,102 [MainThread  ] [INFO ]  using 29797470 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


In [ ]:
bigram.save('../data/lingvo/bigram')

2017-10-07 00:26:05,109 [MainThread  ] [INFO ]  saving Phrases object under ../data/lingvo/bigram, separately None
2017-10-07 00:27:10,345 [MainThread  ] [INFO ]  saved ../data/lingvo/bigram


In [ ]:
for phrase, score in islice(bigram.export_phrases(iter_sents()), 100):
    print('{0}   {1}'.format(phrase, score))

ядерный реактор   12.1388211579
ядерный реактор   12.1388211579
ядерный реактор   12.1388211579
ядерный реактор   12.1388211579
тяжелый фашина   50.9648969508
биопозитивный конструкция   13.9085867073
гидротехнический мелиоративный   123.502952399
природоохранный берегоукрепительный   1163.59214449
защитно-регуляционный сооружение   91.339540658
русло река   198.872468138
тяжелый фашина   50.9648969508
фашина уложить   29.2159597911
плодородный грунт   17.9436537448
многолетний трава   105.072741809
кустарник развитый   23.080766967
ручка-петля укладка   13.5110673198
срок служба   52.3732919658
гидротехнический мелиоративный   123.502952399
природоохранный берегоукрепительный   1163.59214449
русло река   198.872468138
защитно-регуляционный сооружение   91.339540658
тяжелый фашина   50.9648969508
прибрежный откос   11.9251940697
крупнозернистый заполнитель   16.5753433495
срок служба   52.3732919658
1,5-2 м/с   15.5293774132
слабый зарастаемость   58.3902556025
биопозитивность конструк

In [ ]:
bigram = gensim.models.phrases.Phrases.load('../data/lingvo/bigram')

In [ ]:
bigram_ph = gensim.models.phrases.Phraser(bigram)

2017-10-07 00:27:10,386 [MainThread  ] [INFO ]  source_vocab length 29797470
2017-10-07 00:27:23,730 [MainThread  ] [INFO ]  Phraser added 50000 phrasegrams
2017-10-07 00:27:37,104 [MainThread  ] [INFO ]  Phraser added 100000 phrasegrams
2017-10-07 00:27:50,210 [MainThread  ] [INFO ]  Phraser added 150000 phrasegrams
2017-10-07 00:28:03,203 [MainThread  ] [INFO ]  Phraser added 200000 phrasegrams
2017-10-07 00:28:16,085 [MainThread  ] [INFO ]  Phraser added 250000 phrasegrams
2017-10-07 00:28:29,500 [MainThread  ] [INFO ]  Phraser added 300000 phrasegrams
2017-10-07 00:28:42,873 [MainThread  ] [INFO ]  Phraser added 350000 phrasegrams
2017-10-07 00:28:55,730 [MainThread  ] [INFO ]  Phraser added 400000 phrasegrams
2017-10-07 00:29:08,601 [MainThread  ] [INFO ]  Phraser added 450000 phrasegrams
2017-10-07 00:29:21,548 [MainThread  ] [INFO ]  Phraser added 500000 phrasegrams
2017-10-07 00:29:34,627 [MainThread  ] [INFO ]  Phraser added 550000 phrasegrams
2017-10-07 00:29:48,385 [MainThre

In [ ]:
bigram_ph.save('../data/lingvo/bigram_ph')

2017-10-07 00:34:17,775 [MainThread  ] [INFO ]  saving Phraser object under ../data/lingvo/bigram_ph, separately None
2017-10-07 00:34:20,066 [MainThread  ] [INFO ]  saved ../data/lingvo/bigram_ph


In [ ]:
bigram_ph = gensim.models.phrases.Phraser.load('../data/lingvo/bigram_ph')

In [ ]:
trigram = gensim.models.Phrases(bigram_ph[iter_sents()])

2017-10-07 00:34:20,075 [MainThread  ] [INFO ]  collecting all words and their counts
2017-10-07 00:34:20,078 [MainThread  ] [INFO ]  PROGRESS: at sentence #0, processed 0 words and 0 word types
2017-10-07 00:34:21,094 [MainThread  ] [INFO ]  PROGRESS: at sentence #10000, processed 116701 words and 71585 word types
2017-10-07 00:34:22,032 [MainThread  ] [INFO ]  PROGRESS: at sentence #20000, processed 227873 words and 133636 word types
2017-10-07 00:34:23,158 [MainThread  ] [INFO ]  PROGRESS: at sentence #30000, processed 355421 words and 199331 word types
2017-10-07 00:34:24,158 [MainThread  ] [INFO ]  PROGRESS: at sentence #40000, processed 468669 words and 260108 word types
2017-10-07 00:34:25,238 [MainThread  ] [INFO ]  PROGRESS: at sentence #50000, processed 594771 words and 318608 word types
2017-10-07 00:34:26,313 [MainThread  ] [INFO ]  PROGRESS: at sentence #60000, processed 717072 words and 378565 word types
2017-10-07 00:34:27,227 [MainThread  ] [INFO ]  PROGRESS: at sentenc

2017-10-07 00:35:23,862 [MainThread  ] [INFO ]  PROGRESS: at sentence #650000, processed 7232839 words and 3096810 word types
2017-10-07 00:35:24,929 [MainThread  ] [INFO ]  PROGRESS: at sentence #660000, processed 7354004 words and 3132464 word types
2017-10-07 00:35:25,900 [MainThread  ] [INFO ]  PROGRESS: at sentence #670000, processed 7466979 words and 3175734 word types
2017-10-07 00:35:26,860 [MainThread  ] [INFO ]  PROGRESS: at sentence #680000, processed 7577596 words and 3216666 word types
2017-10-07 00:35:27,813 [MainThread  ] [INFO ]  PROGRESS: at sentence #690000, processed 7685375 words and 3254352 word types
2017-10-07 00:35:28,721 [MainThread  ] [INFO ]  PROGRESS: at sentence #700000, processed 7789044 words and 3291180 word types
2017-10-07 00:35:29,705 [MainThread  ] [INFO ]  PROGRESS: at sentence #710000, processed 7900909 words and 3331699 word types
2017-10-07 00:35:30,671 [MainThread  ] [INFO ]  PROGRESS: at sentence #720000, processed 8011336 words and 3374582 wor

2017-10-07 00:36:27,776 [MainThread  ] [INFO ]  PROGRESS: at sentence #1300000, processed 14453341 words and 5591879 word types
2017-10-07 00:36:29,458 [MainThread  ] [INFO ]  PROGRESS: at sentence #1310000, processed 14567216 words and 5630941 word types
2017-10-07 00:36:30,475 [MainThread  ] [INFO ]  PROGRESS: at sentence #1320000, processed 14684749 words and 5666060 word types
2017-10-07 00:36:31,684 [MainThread  ] [INFO ]  PROGRESS: at sentence #1330000, processed 14805186 words and 5705435 word types
2017-10-07 00:36:32,784 [MainThread  ] [INFO ]  PROGRESS: at sentence #1340000, processed 14927093 words and 5739957 word types
2017-10-07 00:36:33,736 [MainThread  ] [INFO ]  PROGRESS: at sentence #1350000, processed 15031906 words and 5774655 word types
2017-10-07 00:36:34,756 [MainThread  ] [INFO ]  PROGRESS: at sentence #1360000, processed 15148299 words and 5812999 word types
2017-10-07 00:36:35,700 [MainThread  ] [INFO ]  PROGRESS: at sentence #1370000, processed 15258242 words

2017-10-07 00:37:38,970 [MainThread  ] [INFO ]  PROGRESS: at sentence #1950000, processed 21777770 words and 7867681 word types
2017-10-07 00:37:39,987 [MainThread  ] [INFO ]  PROGRESS: at sentence #1960000, processed 21895158 words and 7900433 word types
2017-10-07 00:37:40,962 [MainThread  ] [INFO ]  PROGRESS: at sentence #1970000, processed 22009121 words and 7933653 word types
2017-10-07 00:37:41,952 [MainThread  ] [INFO ]  PROGRESS: at sentence #1980000, processed 22118967 words and 7965734 word types
2017-10-07 00:37:42,912 [MainThread  ] [INFO ]  PROGRESS: at sentence #1990000, processed 22225868 words and 8000229 word types
2017-10-07 00:37:43,842 [MainThread  ] [INFO ]  PROGRESS: at sentence #2000000, processed 22333170 words and 8032804 word types
2017-10-07 00:37:44,836 [MainThread  ] [INFO ]  PROGRESS: at sentence #2010000, processed 22446023 words and 8065366 word types
2017-10-07 00:37:45,820 [MainThread  ] [INFO ]  PROGRESS: at sentence #2020000, processed 22553548 words

2017-10-07 00:38:42,922 [MainThread  ] [INFO ]  PROGRESS: at sentence #2600000, processed 28992090 words and 9927464 word types
2017-10-07 00:38:43,878 [MainThread  ] [INFO ]  PROGRESS: at sentence #2610000, processed 29100126 words and 9957550 word types
2017-10-07 00:38:44,932 [MainThread  ] [INFO ]  PROGRESS: at sentence #2620000, processed 29214114 words and 9986102 word types
2017-10-07 00:38:45,840 [MainThread  ] [INFO ]  PROGRESS: at sentence #2630000, processed 29318795 words and 10014005 word types
2017-10-07 00:38:46,886 [MainThread  ] [INFO ]  PROGRESS: at sentence #2640000, processed 29440190 words and 10047050 word types
2017-10-07 00:38:47,902 [MainThread  ] [INFO ]  PROGRESS: at sentence #2650000, processed 29558814 words and 10078744 word types
2017-10-07 00:38:48,814 [MainThread  ] [INFO ]  PROGRESS: at sentence #2660000, processed 29664324 words and 10106755 word types
2017-10-07 00:38:49,803 [MainThread  ] [INFO ]  PROGRESS: at sentence #2670000, processed 29777786 w

2017-10-07 00:39:47,430 [MainThread  ] [INFO ]  PROGRESS: at sentence #3240000, processed 36131856 words and 11863368 word types
2017-10-07 00:39:48,401 [MainThread  ] [INFO ]  PROGRESS: at sentence #3250000, processed 36243868 words and 11893620 word types
2017-10-07 00:39:49,334 [MainThread  ] [INFO ]  PROGRESS: at sentence #3260000, processed 36349023 words and 11923141 word types
2017-10-07 00:39:50,261 [MainThread  ] [INFO ]  PROGRESS: at sentence #3270000, processed 36454191 words and 11951073 word types
2017-10-07 00:39:51,197 [MainThread  ] [INFO ]  PROGRESS: at sentence #3280000, processed 36561488 words and 11977277 word types
2017-10-07 00:39:52,108 [MainThread  ] [INFO ]  PROGRESS: at sentence #3290000, processed 36661148 words and 12007069 word types
2017-10-07 00:39:53,097 [MainThread  ] [INFO ]  PROGRESS: at sentence #3300000, processed 36769921 words and 12035870 word types
2017-10-07 00:39:54,086 [MainThread  ] [INFO ]  PROGRESS: at sentence #3310000, processed 3687950

In [ ]:
trigram.save('../data/lingvo/trigram')

In [ ]:
trigram = gensim.models.phrases.Phrases.load(join(DATA_FOLDER, '../data/lingvo/trigram'))

In [ ]:
for s in islice(trigram[bigram_ph[iter_sents()]], 1000):
    for w in s:
        if len(w.split('_')) > 2:
            print(w)

In [ ]:
trigram_ph = gensim.models.phrases.Phraser(trigram)
trigram_ph.save('../data/lingvo/trigram_ph')

In [ ]:
model = Word2Vec(Sentences(), size=dim, sg=1, min_count=5, window=8, workers=cpu_count)

2017-10-07 05:36:21,253 [MainThread  ] [INFO ]  collecting all words and their counts
2017-10-07 05:36:21,256 [MainThread  ] [INFO ]  PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-10-07 05:36:21,434 [MainThread  ] [INFO ]  PROGRESS: at sentence #10000, processed 132059 words, keeping 13496 word types
2017-10-07 05:36:21,565 [MainThread  ] [INFO ]  PROGRESS: at sentence #20000, processed 254236 words, keeping 21871 word types
2017-10-07 05:36:21,714 [MainThread  ] [INFO ]  PROGRESS: at sentence #30000, processed 400912 words, keeping 30604 word types
2017-10-07 05:36:21,840 [MainThread  ] [INFO ]  PROGRESS: at sentence #40000, processed 523348 words, keeping 37490 word types
2017-10-07 05:36:21,985 [MainThread  ] [INFO ]  PROGRESS: at sentence #50000, processed 664973 words, keeping 42951 word types
2017-10-07 05:36:22,124 [MainThread  ] [INFO ]  PROGRESS: at sentence #60000, processed 799556 words, keeping 48688 word types
2017-10-07 05:36:22,250 [MainThread  ]

2017-10-07 05:36:29,631 [MainThread  ] [INFO ]  PROGRESS: at sentence #630000, processed 7693868 words, keeping 264096 word types
2017-10-07 05:36:29,760 [MainThread  ] [INFO ]  PROGRESS: at sentence #640000, processed 7808454 words, keeping 266567 word types
2017-10-07 05:36:29,891 [MainThread  ] [INFO ]  PROGRESS: at sentence #650000, processed 7926933 words, keeping 269603 word types
2017-10-07 05:36:30,037 [MainThread  ] [INFO ]  PROGRESS: at sentence #660000, processed 8068470 words, keeping 271707 word types
2017-10-07 05:36:30,171 [MainThread  ] [INFO ]  PROGRESS: at sentence #670000, processed 8190606 words, keeping 274787 word types
2017-10-07 05:36:30,311 [MainThread  ] [INFO ]  PROGRESS: at sentence #680000, processed 8311393 words, keeping 278084 word types
2017-10-07 05:36:30,439 [MainThread  ] [INFO ]  PROGRESS: at sentence #690000, processed 8427738 words, keeping 279924 word types
2017-10-07 05:36:30,566 [MainThread  ] [INFO ]  PROGRESS: at sentence #700000, processed 8

2017-10-07 05:36:38,259 [MainThread  ] [INFO ]  PROGRESS: at sentence #1260000, processed 15350970 words, keeping 443310 word types
2017-10-07 05:36:38,398 [MainThread  ] [INFO ]  PROGRESS: at sentence #1270000, processed 15470491 words, keeping 445963 word types
2017-10-07 05:36:38,551 [MainThread  ] [INFO ]  PROGRESS: at sentence #1280000, processed 15588890 words, keeping 447960 word types
2017-10-07 05:36:38,679 [MainThread  ] [INFO ]  PROGRESS: at sentence #1290000, processed 15699404 words, keeping 450951 word types
2017-10-07 05:36:38,828 [MainThread  ] [INFO ]  PROGRESS: at sentence #1300000, processed 15832295 words, keeping 453981 word types
2017-10-07 05:36:38,972 [MainThread  ] [INFO ]  PROGRESS: at sentence #1310000, processed 15954695 words, keeping 456492 word types
2017-10-07 05:36:39,116 [MainThread  ] [INFO ]  PROGRESS: at sentence #1320000, processed 16082510 words, keeping 458566 word types
2017-10-07 05:36:39,275 [MainThread  ] [INFO ]  PROGRESS: at sentence #13300

2017-10-07 05:36:58,207 [MainThread  ] [INFO ]  PROGRESS: at sentence #1890000, processed 23117298 words, keeping 612971 word types
2017-10-07 05:36:58,356 [MainThread  ] [INFO ]  PROGRESS: at sentence #1900000, processed 23241836 words, keeping 615353 word types
2017-10-07 05:36:58,502 [MainThread  ] [INFO ]  PROGRESS: at sentence #1910000, processed 23363261 words, keeping 617725 word types
2017-10-07 05:36:58,654 [MainThread  ] [INFO ]  PROGRESS: at sentence #1920000, processed 23485246 words, keeping 619483 word types
2017-10-07 05:36:58,816 [MainThread  ] [INFO ]  PROGRESS: at sentence #1930000, processed 23623115 words, keeping 621766 word types
2017-10-07 05:36:58,971 [MainThread  ] [INFO ]  PROGRESS: at sentence #1940000, processed 23751927 words, keeping 624755 word types
2017-10-07 05:36:59,126 [MainThread  ] [INFO ]  PROGRESS: at sentence #1950000, processed 23884672 words, keeping 626185 word types
2017-10-07 05:36:59,279 [MainThread  ] [INFO ]  PROGRESS: at sentence #19600

2017-10-07 05:37:07,851 [MainThread  ] [INFO ]  PROGRESS: at sentence #2520000, processed 30818531 words, keeping 743849 word types
2017-10-07 05:37:08,008 [MainThread  ] [INFO ]  PROGRESS: at sentence #2530000, processed 30941734 words, keeping 748188 word types
2017-10-07 05:37:08,168 [MainThread  ] [INFO ]  PROGRESS: at sentence #2540000, processed 31065261 words, keeping 750250 word types
2017-10-07 05:37:08,323 [MainThread  ] [INFO ]  PROGRESS: at sentence #2550000, processed 31184802 words, keeping 751901 word types
2017-10-07 05:37:08,487 [MainThread  ] [INFO ]  PROGRESS: at sentence #2560000, processed 31312178 words, keeping 753995 word types
2017-10-07 05:37:08,637 [MainThread  ] [INFO ]  PROGRESS: at sentence #2570000, processed 31426182 words, keeping 755497 word types
2017-10-07 05:37:08,794 [MainThread  ] [INFO ]  PROGRESS: at sentence #2580000, processed 31548122 words, keeping 756982 word types
2017-10-07 05:37:08,938 [MainThread  ] [INFO ]  PROGRESS: at sentence #25900

...

2017-10-07 05:40:22,010 [MainThread  ] [INFO ]  PROGRESS: at sentence #12420000, processed 151024951 words, keeping 2376199 word types
2017-10-07 05:40:22,207 [MainThread  ] [INFO ]  PROGRESS: at sentence #12430000, processed 151147925 words, keeping 2377822 word types
2017-10-07 05:40:22,403 [MainThread  ] [INFO ]  PROGRESS: at sentence #12440000, processed 151276937 words, keeping 2378775 word types
2017-10-07 05:40:22,604 [MainThread  ] [INFO ]  PROGRESS: at sentence #12450000, processed 151411953 words, keeping 2380423 word types
2017-10-07 05:40:22,788 [MainThread  ] [INFO ]  PROGRESS: at sentence #12460000, processed 151531606 words, keeping 2381741 word types
2017-10-07 05:40:22,995 [MainThread  ] [INFO ]  PROGRESS: at sentence #12470000, processed 151673995 words, keeping 2382906 word types
2017-10-07 05:40:23,180 [MainThread  ] [INFO ]  PROGRESS: at sentence #12480000, processed 151792349 words, keeping 2384025 word types
2017-10-07 05:40:23,373 [MainThread  ] [INFO ]  PROGRES

2017-10-07 05:40:33,707 [MainThread  ] [INFO ]  PROGRESS: at sentence #13030000, processed 158509237 words, keeping 2461956 word types
2017-10-07 05:40:33,909 [MainThread  ] [INFO ]  PROGRESS: at sentence #13040000, processed 158632175 words, keeping 2463056 word types
2017-10-07 05:40:34,126 [MainThread  ] [INFO ]  PROGRESS: at sentence #13050000, processed 158772779 words, keeping 2464236 word types
2017-10-07 05:40:34,313 [MainThread  ] [INFO ]  PROGRESS: at sentence #13060000, processed 158887285 words, keeping 2465732 word types
2017-10-07 05:40:34,508 [MainThread  ] [INFO ]  PROGRESS: at sentence #13070000, processed 159004733 words, keeping 2467282 word types
2017-10-07 05:40:34,745 [MainThread  ] [INFO ]  PROGRESS: at sentence #13080000, processed 159125603 words, keeping 2469232 word types
2017-10-07 05:40:34,906 [MainThread  ] [INFO ]  PROGRESS: at sentence #13090000, processed 159250455 words, keeping 2470895 word types
2017-10-07 05:40:35,088 [MainThread  ] [INFO ]  PROGRES

2017-10-07 05:40:45,200 [MainThread  ] [INFO ]  PROGRESS: at sentence #13640000, processed 165854849 words, keeping 2548154 word types
2017-10-07 05:40:45,381 [MainThread  ] [INFO ]  PROGRESS: at sentence #13650000, processed 165966451 words, keeping 2549250 word types
2017-10-07 05:40:45,565 [MainThread  ] [INFO ]  PROGRESS: at sentence #13660000, processed 166083268 words, keeping 2550415 word types
2017-10-07 05:40:45,755 [MainThread  ] [INFO ]  PROGRESS: at sentence #13670000, processed 166203698 words, keeping 2551737 word types
2017-10-07 05:40:45,947 [MainThread  ] [INFO ]  PROGRESS: at sentence #13680000, processed 166329733 words, keeping 2552639 word types
2017-10-07 05:40:46,138 [MainThread  ] [INFO ]  PROGRESS: at sentence #13690000, processed 166450702 words, keeping 2554160 word types
2017-10-07 05:40:46,320 [MainThread  ] [INFO ]  PROGRESS: at sentence #13700000, processed 166570758 words, keeping 2555407 word types
2017-10-07 05:40:46,501 [MainThread  ] [INFO ]  PROGRES

2017-10-07 05:40:56,720 [MainThread  ] [INFO ]  PROGRESS: at sentence #14250000, processed 173262511 words, keeping 2629226 word types
2017-10-07 05:40:56,908 [MainThread  ] [INFO ]  PROGRESS: at sentence #14260000, processed 173385546 words, keeping 2630514 word types
2017-10-07 05:40:57,112 [MainThread  ] [INFO ]  PROGRESS: at sentence #14270000, processed 173521883 words, keeping 2632286 word types
2017-10-07 05:40:57,300 [MainThread  ] [INFO ]  PROGRESS: at sentence #14280000, processed 173641844 words, keeping 2633660 word types
2017-10-07 05:40:57,507 [MainThread  ] [INFO ]  PROGRESS: at sentence #14290000, processed 173769420 words, keeping 2634759 word types
2017-10-07 05:40:57,677 [MainThread  ] [INFO ]  PROGRESS: at sentence #14300000, processed 173880798 words, keeping 2636258 word types
2017-10-07 05:40:57,887 [MainThread  ] [INFO ]  PROGRESS: at sentence #14310000, processed 174016444 words, keeping 2637759 word types
2017-10-07 05:40:58,071 [MainThread  ] [INFO ]  PROGRES

2017-10-07 05:41:08,352 [MainThread  ] [INFO ]  PROGRESS: at sentence #14860000, processed 180688570 words, keeping 2708493 word types
2017-10-07 05:41:08,540 [MainThread  ] [INFO ]  PROGRESS: at sentence #14870000, processed 180807523 words, keeping 2709620 word types
2017-10-07 05:41:08,735 [MainThread  ] [INFO ]  PROGRESS: at sentence #14880000, processed 180934348 words, keeping 2710923 word types
2017-10-07 05:41:08,935 [MainThread  ] [INFO ]  PROGRESS: at sentence #14890000, processed 181064673 words, keeping 2712288 word types
2017-10-07 05:41:09,130 [MainThread  ] [INFO ]  PROGRESS: at sentence #14900000, processed 181191808 words, keeping 2713554 word types
2017-10-07 05:41:09,323 [MainThread  ] [INFO ]  PROGRESS: at sentence #14910000, processed 181313528 words, keeping 2714762 word types
2017-10-07 05:41:09,510 [MainThread  ] [INFO ]  PROGRESS: at sentence #14920000, processed 181430106 words, keeping 2716183 word types
2017-10-07 05:41:09,730 [MainThread  ] [INFO ]  PROGRES

In [ ]:
model.save('../data/lingvo/w2v_300_sg_5_w8')